In [4]:
import pickle
import pandas as pd
import numpy as np
import datetime as dt
import requests
from bs4 import BeautifulSoup
import re
import multiprocess
import warnings
warnings.filterwarnings("ignore")

In [2]:
# datetime of each financial report
with open('../fp_url_htm/company_dt_dic_final.pkl','rb') as f:
    company_dt_dic = pickle.load(f)
    
# htm or txt url of each financial report
with open('../fp_url_htm/company_fp_dic_final.pkl','rb') as f:
    company_fp_dic = pickle.load(f)
    
# mark of each financial report
# 0 : crawl from raw 1(htm or txt)  1: crawl from original txt
with open('../fp_url_htm/company_mk_dic_final.pkl','rb') as f:
    company_mk_dic = pickle.load(f)

In [3]:
SP_return=pd.read_csv("SP_return.csv",index_col=0,parse_dates=True)
SP_price=(SP_return+1).cumprod()["sprtrn"]

In [5]:
all_company_data=pd.read_csv("all_company_data.csv")

In [27]:
def cut_item2(input_data_):
    input_data, idx = input_data_
    output=input_data.copy()
    output["text"]="NA"
    else_deal=[]
    for i in range(input_data.shape[0]):
        try:
#             print("start")
            index_now=input_data.index[i]
            url="https://www.sec.gov"+input_data["url"].iloc[i]
            res = requests.get(url)
            soup = BeautifulSoup(res.text, 'html.parser')
            soup_txt = soup.get_text()
            soup_txt=soup_txt.replace('\xa0',' ').replace('&nbsp;',' ').replace('&#160;',' ').replace('\n',' ').replace('\x93',' ').replace('\x94',' ')
            soup_txt=soup_txt.split(" ")
            while '' in soup_txt:
                soup_txt.remove('')
            space=" "
            soup_txt=space.join(soup_txt)
            soup_txt=soup_txt.lower()
        #    if i%10 == 0:
        #        print(i)
            pattern=re.compile("item 2. manage")
            result_iter1=pattern.finditer(soup_txt)
            result_item2 = [ m1.span() for m1 in result_iter1]
            if len(result_item2) == 0:
                pattern=re.compile("item 2 – manage")
                result_iter1=pattern.finditer(soup_txt)
                result_item2 = [ m1.span() for m1 in result_iter1]
            if len(result_item2) == 0:
                pattern=re.compile("item 2.manage")
                result_iter1=pattern.finditer(soup_txt)
                result_item2 = [ m1.span() for m1 in result_iter1]
            pattern=re.compile("item 3. quant")
            result_iter1=pattern.finditer(soup_txt)
            result_item3 = [ m1.span() for m1 in result_iter1]
            pattern=re.compile("part ii")
            result_iter1=pattern.finditer(soup_txt)
            result_part2=[ m1.span() for m1 in result_iter1]
            if len(result_item2) == 2:
                if len(result_item3) == 2:
                    output["text"].loc[index_now]=soup_txt[result_item2[1][0]:result_item3[1][0]]
                else:
                    output["text"].loc[index_now]=soup_txt[result_item2[1][0]:result_part2[1][0]]
            elif len(result_item2) == 1:
                output["text"].loc[index_now]=soup_txt[result_item2[0][0]:result_part2[0][0]]
            elif len(result_item2) > 3:
                output["text"].loc[index_now]=soup_txt[result_item2[1][0]:result_item3[1][0]]
            elif len(result_item2) == 3:
                output["text"].loc[index_now]=soup_txt[result_item2[-1][0]:result_item3[-1][0]]
            else:
                print(len(result_item2))
                print(url)
                else_deal.append(i)
                print(i)
        except:
            print("unsolved")
            print(i)
            
    output.to_csv(f'cut_data_{idx}.csv')
    return output

In [6]:
all_company_data.head()

,company,date,return,bad,url
0,A,2000-03-15,0.680463,1.0,/Archives/edgar/data/1090872/00009120570001170...
1,A,2000-06-12,0.937685,0.0,/Archives/edgar/data/1090872/00009120570002820...
2,A,2001-03-19,0.947393,0.0,/Archives/edgar/data/1090872/00010958110100163...
3,A,2001-06-14,0.821604,0.0,/Archives/edgar/data/1090872/00008916180150118...
4,A,2002-03-06,0.741458,0.0,/Archives/edgar/data/1090872/00008916180200104...


In [7]:
length = int(all_company_data.shape[0]/4)
company_data0, company_data1, company_data2, company_data3 = all_company_data.iloc[:length,:], all_company_data.iloc[length:length*2,:],\
                                            all_company_data.iloc[length*2:length*3,:],all_company_data.iloc[length*3:,:]

In [8]:
works = [(company_data0,0),(company_data1,1),(company_data2,2),(company_data3,3)]
works

[(      company        date    return  bad  \
  0           A  2000-03-15  0.680463  1.0   
  1           A  2000-06-12  0.937685  0.0   
  2           A  2001-03-19  0.947393  0.0   
  3           A  2001-06-14  0.821604  0.0   
  4           A  2002-03-06  0.741458  0.0   
  5           A  2002-06-05  0.560176  1.0   
  6           A  2003-03-12  1.192572  0.0   
  7           A  2003-06-04  1.312234  0.0   
  8           A  2004-03-04  0.805133  1.0   
  9           A  2004-06-03  0.823663  0.0   
  10          A  2005-03-10  1.061255  0.0   
  11          A  2005-06-06  1.312659  0.0   
  12          A  2006-03-09  0.908075  0.0   
  13          A  2006-06-06  0.981595  0.0   
  14          A  2007-03-09  1.208834  0.0   
  15          A  2007-06-05  0.966714  0.0   
  16          A  2008-03-07  1.217379  0.0   
  17          A  2008-06-05  0.958895  0.0   
  18          A  2009-03-12  1.345492  0.0   
  19          A  2009-06-09  1.353392  0.0   
  20          A  2010-03-10  0.968

In [ ]:
if __name__ == '__main__':
    ans = []
    pool = multiprocess.Pool(processes=4)
    ans.append(pool.map(cut_item2,works))

unsolved
79
unsolved
80
unsolved
81
unsolved
82
unsolved
83
unsolved
84
unsolved
85
unsolved
86
unsolved
87
unsolved
88
unsolved
89
unsolved
91
unsolved
92
unsolved
93
unsolved
94
unsolved
95
unsolved
96
unsolved
97
unsolved
98
unsolved
99
unsolved
100
0
https://www.sec.gov/Archives/edgar/data/355948/0000355948-94-000004.txt
118
0
https://www.sec.gov/Archives/edgar/data/355948/0000355948-94-000024.txt
119
unsolved
101
0
https://www.sec.gov/Archives/edgar/data/355948/0000355948-95-000002.txt
120
0
https://www.sec.gov/Archives/edgar/data/355948/0000355948-95-000015.txt
121
unsolved
102
0
https://www.sec.gov/Archives/edgar/data/355948/0000355948-96-000003.txt
122
0
https://www.sec.gov/Archives/edgar/data/355948/0000355948-96-000027.txt
123
unsolved
103
0
https://www.sec.gov/Archives/edgar/data/355948/0000355948-97-000002.txt
124
0
https://www.sec.gov/Archives/edgar/data/355948/0000355948-97-000026.txt
125
0
https://www.sec.gov/Archives/edgar/data/355948/0000355948-98-000002.txt
126
unsolv

403
0
https://www.sec.gov/Archives/edgar/data/811212/0001009529-96-000015.txt
502
0
https://www.sec.gov/Archives/edgar/data/811212/0000811212-97-000002.txt
503
unsolved
290
unsolved
291
0
https://www.sec.gov/ix?doc=/Archives/edgar/data/27904/000002790417000008/dal331201710q.htm
313
0
https://www.sec.gov/ix?doc=/Archives/edgar/data/27904/000002790417000013/dal630201710q.htm
314
0
https://www.sec.gov/ix?doc=/Archives/edgar/data/27904/000002790418000013/dal331201810q.htm
315
0
https://www.sec.gov/ix?doc=/Archives/edgar/data/27904/000002790418000016/dal630201810q.htm
316
unsolved
323
0
https://www.sec.gov/Archives/edgar/data/1018952/0000950170-98-000963.txt
644
0
https://www.sec.gov/Archives/edgar/data/1018952/0000950170-98-001706.txt
645
0
https://www.sec.gov/Archives/edgar/data/1018952/0000950170-99-000784.txt
646
0
https://www.sec.gov/Archives/edgar/data/1018952/000095017099001242/0000950170-99-001242.txt
647
0
https://www.sec.gov/Archives/edgar/data/1018952/000095017000000772/000095017

860
0
https://www.sec.gov/Archives/edgar/data/4611/0000004611-97-000012.txt
861
0
https://www.sec.gov/Archives/edgar/data/4611/0000004611-97-000017.txt
862
0
https://www.sec.gov/Archives/edgar/data/4611/0001005477-98-001510.txt
863
0
https://www.sec.gov/Archives/edgar/data/4611/0001005477-98-002458.txt
864
0
https://www.sec.gov/Archives/edgar/data/4611/0001005477-99-002293.txt
865
0
https://www.sec.gov/Archives/edgar/data/4611/000100547799003616/0001005477-99-003616.txt
866
0
https://www.sec.gov/Archives/edgar/data/4611/000100547700003850/0001005477-00-003850.txt
867
0
https://www.sec.gov/Archives/edgar/data/4611/000100547700005491/0001005477-00-005491-0001.txt
868
unsolved
1103
unsolved
951
unsolved
952
unsolved
953
unsolved
954
unsolved
955
unsolved
956
unsolved
957
unsolved
958
unsolved
959
unsolved
715
unsolved
1104
unsolved
1105
0
https://www.sec.gov/Archives/edgar/data/1049480/000104746999031813/0001047469-99-031813.txt
716
unsolved
1107
0
https://www.sec.gov/Archives/edgar/data/

In [32]:
output = pd.concat([ans[0][0],ans[0][1],ans[0][2],ans[0][3]],axis = 0)

In [43]:
output

,company,date,return,bad,url,text
0,A,2000-03-15,0.680463,1.0,/Archives/edgar/data/1090872/00009120570001170...,item 2. management's discussion and analysis o...
1,A,2000-06-12,0.937685,0.0,/Archives/edgar/data/1090872/00009120570002820...,item 2. management's discussion and analysis o...
2,A,2001-03-19,0.947393,0.0,/Archives/edgar/data/1090872/00010958110100163...,item 2. management's discussion and analysis o...
3,A,2001-06-14,0.821604,0.0,/Archives/edgar/data/1090872/00008916180150118...,item 2. management's discussion and analysis o...
4,A,2002-03-06,0.741458,0.0,/Archives/edgar/data/1090872/00008916180200104...,item 2. managements discussion and analysis o...
5,A,2002-06-05,0.560176,1.0,/Archives/edgar/data/1090872/00008916180200270...,item 2. managements discussion and analysis o...
6,A,2003-03-12,1.192572,0.0,/Archives/edgar/data/1090872/00008916180300121...,item 2. managements discussion and analysis o...
7,A,2003-06-04,1.312234,0.0,/Archives/edgar/data/1090872/00008916180300285...,item 2. managements discussion and analysis o...
8,A,2004-03-04,0.805133,1.0,/Archives/edgar/data/1090872/00011931250403472...,item 2. managements discussion and analysis o...
9,A,2004-06-03,0.823663,0.0,/Archives/edgar/data/1090872/00011931250409832...,item 2. managements discussion and analysis o...
